In [1]:
# ------------------------------------------------------------------
# Initial import.
# ------------------------------------------------------------------
import numpy as np
import os
import pandas as pd
import sys
import cv2
from tpot import TPOTClassifier

In [106]:
# ------------------------------------------------------------------
# Create a function to load the data(Actually, it is the directory of data).
# So we can visit those data easily by visiting their address.
# ------------------------------------------------------------------

# Firstly declare an address. This address is prepared for the next step.
PATH = "/home/mlvm2/ce888labs/ce888_ass/Japanese_(hiragana)"

# Create a list of items in the address we declared above.
Charac_list = list(os.path.join(PATH, Charac) for Charac in os.listdir(PATH))

# print(Charac_list)

# Function to load data.
# Two images(images' address) should be included
# Compare their differnce and label 0 or 1(0 for differnce, 1 for same).
# Return a list included address and difference labal.
# use them as training samples. 
def Load_data():
    data = ["", "", 0] # first, create a list including messages we need to return.
    rand_charac_prev = [0, 0] # rand_charac_prev is prepared for comparing the difference betwwen two data
                              # simpy comparing the random number.
    for i in range(0, 2):
        rand_charac = np.random.randint(52) # random choose.
        rand_letter = np.random.randint(20) # random choose.
        rand_charac_path = Charac_list[rand_charac]
        Letters_list = list(os.path.join(rand_charac_path, letter) for letter in os.listdir(rand_charac_path))
        data[i] = Letters_list[rand_letter]
        rand_charac_prev[i] = rand_charac
    if rand_charac_prev[0] != rand_charac_prev[1]: # if they are in the same folder, then they are the same,else not.
        data[2] = 0
    else:
        data[2] = 1
    return data

# A developed fuction only allows character 0 as the ground true.
def Load_data_developed():
    data = ["", "", 0]
    rand_charac_prev = [0, 0]
    for i in range(0, 2):
        rand_charac = np.random.randint(2) # random choose.
        rand_letter = np.random.randint(20) # random choose.
        rand_charac_path = Charac_list[rand_charac]
        Letters_list = list(os.path.join(rand_charac_path, letter) for letter in os.listdir(rand_charac_path))
        data[i] = Letters_list[rand_letter]  
        rand_charac_prev[i] = rand_charac
    if  rand_charac_prev[0] != rand_charac_prev[1]: # if they are in the same folder, then they are the same,else not.
        data[2] = 0
    else:
        data[2] = 1
    return data

# A developed fuction only allows character 0 as the ground true,and provide train data.
def Load_train_data():
    data = ["", "", 0]
    rand_charac_prev = [0, 0]
    for i in range(0, 2):
        rand_charac = 0
        if i==1:
            rand_charac = np.random.randint(2)
            rand_letter = np.random.randint(20) # random choose.
        else:
            rand_letter = 0
        charac_path = Charac_list[rand_charac]
        Letters_list = list(os.path.join(charac_path, letter) for letter in os.listdir(charac_path))
        data[i] = Letters_list[rand_letter]  
        rand_charac_prev[i] = rand_charac
    if  rand_charac_prev[1] != 0: # if they are in the same folder, then they are the same,else not.
        data[2] = 0
    else:
        data[2] = 1
    return data
    
data1 = Load_data()
print(data1[0])

Data_test = [["test image 1", "test image 2", "difference true=0"]]
Data_train = [["train image 1", "train image 2", "difference true=0"]]

for i in range(0,1000): # we randomly get 1000 samples as our test dataset. 
    data1 = Load_data_developed()
    Data_test.append(data1)

for i in range(0,10): # we randomly get 1 samples as our train dataset. 
    data1 = Load_train_data()
    Data_train.append(data1)
    
# print(Data)

df1 = pd.DataFrame(Data_test) 
df2 = pd.DataFrame(Data_train)

df2.iloc[1,1]

/home/mlvm2/ce888labs/ce888_ass/Japanese_(hiragana)/character46/0533_14.png


'/home/mlvm2/ce888labs/ce888_ass/Japanese_(hiragana)/character01/0488_17.png'

In [107]:
# --------------------------------------------------------------------------
# Save our data into a .csv file.
# --------------------------------------------------------------------------
df1.to_csv("Test_data",index='False')
df2.to_csv("Train_data",index='False')

In [108]:
df2

,0,1,2
0,train image 1,train image 2,difference true=0
1,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,1
2,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,0
3,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,0
4,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,0
5,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,0
6,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,1
7,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,0
8,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,1
9,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,0


In [109]:
# ------------------------------------------------------------
# Data read and pre-processing.
# ------------------------------------------------------------
df_train = pd.read_csv("Train_data")
df_train.drop(df_train.index[0], inplace = True)
del df_train["Unnamed: 0"]
df_train
df_test = pd.read_csv("Test_data")
df_test.drop(df_test.index[0], inplace = True)
del df_test["Unnamed: 0"]
df_test

,0,1,2
1,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,0
2,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,1
3,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,1
4,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,0
5,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,1
6,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,1
7,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,0
8,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,0
9,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,0
10,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,/home/mlvm2/ce888labs/ce888_ass/Japanese_(hira...,0


In [110]:
df_test.iloc[0,1]

'/home/mlvm2/ce888labs/ce888_ass/Japanese_(hiragana)/character01/0488_20.png'

In [111]:
X_train_1 = cv2.imread(df_train.iloc[0,0], cv2.IMREAD_GRAYSCALE)
X_train_1 = X_train_1.reshape(11025)
X_train_1

array([255, 255, 255, ..., 255, 255, 255], dtype=uint8)

In [112]:
# ----------------------------------------------------------------------
# A final confirmed data processing method.
# ----------------------------------------------------------------------
X_train_1 = cv2.imread(df_train.iloc[0,0], cv2.IMREAD_GRAYSCALE)
X_train_2 = cv2.imread(df_train.iloc[0,1], cv2.IMREAD_GRAYSCALE)
X_train_wfp = X_train_1,X_train_2
X_train_wfp = np.array(X_train_wfp)
X_train = X_train_wfp.reshape(1, 22050)

for i in range(1, 10):
    X_train_1 = cv2.imread(df_train.iloc[i,0], cv2.IMREAD_GRAYSCALE)
    X_train_2 = cv2.imread(df_train.iloc[i,1], cv2.IMREAD_GRAYSCALE)
    X_train_wfp = X_train_1,X_train_2
    X_train_wfp = np.array(X_train_wfp)
    X_train_wfp = X_train_wfp.reshape(1, 22050)
    X_train = np.r_[X_train, X_train_wfp]

y_train = df_train.iloc[0:10, 2]
y_train = np.array(y_train)

X_test_1 = cv2.imread(df_test.iloc[0,0], cv2.IMREAD_GRAYSCALE)
X_test_2 = cv2.imread(df_test.iloc[0,1], cv2.IMREAD_GRAYSCALE)
X_test_wfp = X_test_1,X_test_2
X_test_wfp = np.array(X_test_wfp)
X_test = X_test_wfp.reshape(1, 22050)

for i in range(1, 1000):
    X_test_1 = cv2.imread(df_test.iloc[i,0], cv2.IMREAD_GRAYSCALE)
    X_test_2 = cv2.imread(df_test.iloc[i,1], cv2.IMREAD_GRAYSCALE)
    X_test_wfp = X_test_1,X_test_2
    X_test_wfp = np.array(X_test_wfp)
    X_test_wfp = X_test_wfp.reshape(1, 22050)
    X_test = np.r_[X_test, X_test_wfp]

y_test = df_test.iloc[0:1000, 2]
y_test = np.array(y_test)

    
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(10, 22050)
(1000, 22050)
(10,)
(1000,)


In [104]:
y_train

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], dtype=object)

In [121]:
ipeline_optimizer = TPOTClassifier(generations=5, population_size=10, verbosity=2)
ipeline_optimizer.fit(X_train, y_train)

Version 0.9.2 of tpot is outdated. Version 0.9.3 was released 3 days ago.


Optimization Progress:  33%|███▎      | 20/60 [00:05<00:20,  1.95pipeline/s]

Generation 1 - Current best internal CV score: 0.766666666667


Optimization Progress:  52%|█████▏    | 31/60 [00:36<02:20,  4.85s/pipeline]

Generation 2 - Current best internal CV score: 1.0


...

Generation 3 - Current best internal CV score: 1.0


Optimization Progress:  83%|████████▎ | 50/60 [00:41<00:05,  1.84pipeline/s]

Generation 4 - Current best internal CV score: 1.0


Generation 5 - Current best internal CV score: 1.0

Best pipeline: GaussianNB(Nystroem(input_matrix, gamma=0.45, kernel=cosine, n_components=1))


TPOTClassifier(config_dict={'sklearn.ensemble.GradientBoostingClassifier': {'max_features': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]), 'learning_rate': [0.001, 0.01, 0.1, 0.5, 1.0], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7...  0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])}, 'sklearn.preprocessing.RobustScaler': {}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=5, max_eval_time_mins=5,
        max_time_mins=None, memory=None, mutation_rate=0.9, n_jobs=1,
        offspring_size=10, periodic_checkpoint_folder=None,
        population_size=10, random_state=None, scoring=None, subsample=1.0,
        verbosity=2, warm_start=False)

In [122]:
ipeline_optimizer.score(X_test, y_test) #score should be high and the result shows it is totally a shit classifier.

/home/mlvm2/.local/lib/python2.7/site-packages/sklearn/metrics/classification.py:181: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


0.0

In [123]:
ipeline_optimizer.export('tpot_ogmnist_pipeline.py')

True

In [259]:
Y_train = np_utils.to_categorical(y_train, 2)
Y_test = np_utils.to_categorical(y_test, 2)

In [11]:
Y_train

NameError: name 'Y_train' is not defined

In [12]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Activation, Embedding, Flatten, Input, Dropout, Conv1D, GlobalMaxPooling1D, recurrent
from keras.datasets import imdb
from keras.utils import np_utils

/home/mlvm2/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [77]:
print('Build model...')


inputs = Input(shape=(22050,))
x = inputs

for i in range(0, 3):
    x = Dense(units=64, activation='relu')(x)

x = Dropout(0.5, noise_shape=None, seed=None)(x)
predictions = Dense(1, activation='softmax')(x)


model = Model(input=inputs, output=predictions)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


/home/mlvm2/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  


In [78]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 22050)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 64)                1411264   
_________________________________________________________________
dense_18 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_19 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 65        
Total params: 1,419,649
Trainable params: 1,419,649
Non-trainable params: 0
_________________________________________________________________


In [116]:
print('Train...')
model.fit(X_train, y_train, batch_size=32, nb_epoch=10,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test,
                            batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 10 samples, validate on 1000 samples
Epoch 1/10


/home/mlvm2/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


10/10 [==============================] - 0s 33ms/step - loss: 11.1597 - acc: 0.3000 - val_loss: 8.1147 - val_acc: 0.4910
Epoch 2/10
10/10 [==============================] - 0s 24ms/step - loss: 11.1597 - acc: 0.3000 - val_loss: 8.1147 - val_acc: 0.4910
Epoch 3/10
10/10 [==============================] - 0s 33ms/step - loss: 11.1597 - acc: 0.3000 - val_loss: 8.1147 - val_acc: 0.4910
Epoch 4/10
10/10 [==============================] - 0s 32ms/step - loss: 11.1597 - acc: 0.3000 - val_loss: 8.1147 - val_acc: 0.4910
Epoch 5/10
10/10 [==============================] - 0s 30ms/step - loss: 11.1597 - acc: 0.3000 - val_loss: 8.1147 - val_acc: 0.4910
Epoch 6/10
10/10 [==============================] - 0s 24ms/step - loss: 11.1597 - acc: 0.3000 - val_loss: 8.1147 - val_acc: 0.4910
Epoch 7/10
10/10 [==============================] - 0s 33ms/step - loss: 11.1597 - acc: 0.3000 - val_loss: 8.1147 - val_acc: 0.4910
Epoch 8/10
10/10 [==============================] - 0s 30ms/step - loss: 11.1597 - acc:

In [113]:
count=0
for i in range(0, 1000):
    j = y_test[i]
    if j == '0':
        count = count+1
print(count)

509


In [114]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(X_train, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [115]:
neigh.score(X_test, y_test)

0.505

In [126]:
import numpy as np
import pandas as pd
from sklearn.kernel_approximation import Nystroem
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline

exported_pipeline = make_pipeline(
    Nystroem(gamma=0.45, kernel="cosine", n_components=1),
    GaussianNB()
)

exported_pipeline.fit(X_train, y_train)
exported_pipeline.score(X_test, y_test)

0.516